In [13]:
from pandas import read_csv, DataFrame

In [14]:
def get_all_data():
    my_file_dir = "merged_all_data.csv"
    df = read_csv(my_file_dir)
    return df


##########################################################################################
def replace_outliers_within_group(df:DataFrame, group_columns:list, target_column:str):
    # Reset index to ensure group_columns are not in the index
    df = df.reset_index(drop=True)

    # Function to calculate and replace outliers within each group
    def replace(group):
        Q1 = group[target_column].quantile(0.25)
        Q3 = group[target_column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        mean_value = group[(group[target_column] >= lower_bound) & (group[target_column] <= upper_bound)][target_column].mean()
        group[target_column] = group[target_column].apply(lambda x: mean_value if x < lower_bound or x > upper_bound else x)
        return group

    # Apply the function to each subgroup
    return df.groupby(group_columns).apply(replace).reset_index(drop=True)

In [15]:
df = get_all_data()
df = df[['pid', 'condition', 'time', 'cond_order']]
df.head(5)

,pid,condition,time,cond_order
0,1,no,52.200,viz_first
1,2,no,105.725,viz_first
2,3,no,121.242,viz_first
3,4,no,188.917,viz_first
4,5,no,85.672,viz_first


In [21]:
cond_order_list = df['cond_order'].tolist()
cond_list = df['condition'].tolist()
order_list = []

for i in range(len(cond_list)):
    cond = cond_list[i]
    cond_order = cond_order_list[i]
    if cond == "no":
        if cond_order == "none_first":
            order_list.append("trial1")
        else:
            order_list.append("trial2")
    else:
        if cond_order == "viz_first":
            order_list.append("trial1")
        else:
            order_list.append("trial2")

df['order'] = order_list

df = replace_outliers_within_group(df, ['condition', 'order'], "time")

# write new dataframe to csv file
dest_path = "order_interaction.csv"
df.to_csv(dest_path, index=False)